In [307]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild


class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
    
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)

        knn = self.nearest[:, 1]
        belong = []
        for i in knn:
            belong.append(i.data[-1])
        b = max(set(belong), key=belong.count)

        return self.nearest, b

In [308]:
import numpy as np
from math import sqrt
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
min_max_scaler=preprocessing.MinMaxScaler()



cmc=pd.read_table('./mydata/contraception.txt',sep=',')
cmc.columns=(['wife_age','wife_edu','husband_edu','num_child','wife_religion','wife_work_station','husband_occupation','living_level','media_exposure','label'])
cmc.drop_duplicates(inplace=True)#删除重复数据
cmc_sample=cmc[['wife_age','wife_edu','husband_edu','num_child','wife_religion','wife_work_station','husband_occupation','living_level','media_exposure']]
cmc_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(cmc_sample)))
cmc_sample['label']=list(cmc.label-1) # 不加list标签会出错

In [309]:

def run(data):  
    print('epoch..........:',term)
    Dn = []
    count2=0
    for x in data:
        #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
        near, belong = kdt.search(x[:-1], 8)  # 设置临近点的个数
        #print("test:")
       # print(x, "predict:", belong)
       # print("nearest:")
        density = 0
        t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
        for n in near:
            if x[-1] == n[1].data[-1] and (x != n[1].data).any():
                t += 1
                hon += 1
                dist_hon += n[0]

            elif  x[-1] != n[1].data[-1]:
                hen += 1
                dist_hen += n[0]

            density += n[0]  #密度   

        DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
        DRL = (hen - hon) / 7  #标签异同差比    
        Ps = t / 7
        Pd = 1 - Ps
        if (Ps==1)|(Ps==0):
            H = 0
        else:
            H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
      #  print('densit:',density)
       
        if (density <= a): #高密度区域

            if DRL > 0:
                Dn.append(list(x))
                print('高密度区噪声')
                print(x)
                for n in near:
                    print(n[1].data,n[0])
        else:
            if (H < H_): #低密度单一标签区域
        
                if DRL > 0:
                    Dn.append(list(x))
                   # print('低密度单一标签噪声')
   
            else: #低密度混合标签区  
        
              #  print('DoD',DoD)
                DoD_ = density / 7
               # print('DoD阈值:',DoD_)
                if DoD >= DoD_:
                    hon_sa,hen_sa,DoD_sa=0,0,0
                    near2=[]
                    for n in near:
                        if n[0] <= a: #该点到待测点距离小于a
                            near2.append(n)
                    for n2 in near2:
                        if (x[-1]==n2[1].data[-1])and (x != n2[1].data).any():
                            hon_sa +=1
                        elif x[-1] !=n2[1].data[-1]:
                            hen_sa +=1
                    DoD_sa = hen_sa - hon_sa
                    if DoD_sa > 0:        
                        Dn.append(list(x))
                      #  print(x)

                    elif DoD_sa ==0 and x[-1] != near[len(near2)][1].data[-1]:
                        Dn.append(list(x))  
                      #  print(x)
                        
    noise_list=noise.values.tolist()
    TP = 0
    for c in Dn:
        if c in noise_list:
            TP += 1
    FP = len(Dn) - TP
    FN = len(noise_list) - TP
    precision= TP / (TP + FP)
    recall= TP / (TP + FN)
    fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
    re = len(Dn) / len(data) #移除率
    false_re= FP / len(data)#错误移除比率
    print('准确率:',precision)
    print('召回率:',recall)
    print('F0.5值:',fh)
    print('移除率:',re)
    return fh,re,false_re

In [310]:
def run2(data):  
    print('epoch..........:',term)
    Dn,Dn_filted = [],[]
     
    for x in data:
        #plt.scatter(x[0], x[1], c='red', marker='x')  # 测试点
        near, belong = kdt.search(x[:-1], k+1)  # 设置临近点的个数
        #print("test:")
       # print(x, "predict:", belong)
       # print("nearest:")
        density = 0
        t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
        for n in near:
            if x[-1] == n[1].data[-1] and (x != n[1].data).any():
                t += 1
                hon += 1
                dist_hon += n[0]

            elif  x[-1] != n[1].data[-1]:
                hen += 1
                dist_hen += n[0]

            density += n[0]  #密度   
        
    
        DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
        DRL = (hen - hon)   #标签异同差 
            
        Ps = t / k
        Pd = 1 - Ps
        if (Ps==1)|(Ps==0):
            H = 0
        else:
            H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
      #  print('densit:',density)
       
        if (density <= a): #高密度区域
            
            if DRL > 0:
                Dn.append(list(x))
        else:
            if (H < H_): #低密度单一标签区域
                if DRL > 0:
                    Dn.append(list(x))
            else: #低密度混合标签区 
                DoD_ =  density / k
               # print('DoD阈值:',DoD_)
                if DoD >= DoD_:
                    hon_sa,hen_sa,DRL_sa=0,0,0
                    near2=[]
                    for n in near:
                        if n[0] <= a: #该点到待测点距离小于a
                            near2.append(n)
                    for n2 in near2:
                        if (x[-1]==n2[1].data[-1])and (x != n2[1].data).any():
                            hon_sa +=1
                        elif x[-1] !=n2[1].data[-1]:
                            hen_sa +=1
                    DRL_sa = hen_sa - hon_sa
                    if DRL_sa > 0:        
                        Dn.append(list(x))

                    elif DRL_sa ==0 and x[-1] != near[len(near2)][1].data[-1]:
                        #print('加入的数据:',near[len(near2)][1].data)
                        Dn.append(list(x))           
    noise_neighbor=[]                   
    for e in Dn:  # 计算噪声分数
        e=np.array(e)
        near3,belong3=kdt.search(e[:-1],k+1) #k近邻
        for n3 in near3:
            noise_neighbor.append(list(n3[1].data))
    for e in Dn:
        e=np.array(e)
        near4,belong4=kdt.search(e[:-1],k+1)
        t_e=noise_neighbor.count(list(e))-1 #处于其它噪声近邻的次数
        confidence_e= 1 / np.sqrt(1+t_e*t_e)#e的置信度
        for i in range(1,k+1):
            t_ei=noise_neighbor.count(list(near4[i][1].data))
            confidence_ei=1 / np.sqrt(1+t_ei*t_ei) 
            if near4[i][1].data[-1]==e[-1]:
                different_class=-1
            else:
                different_class=1
            if list(near4[i][1].data) in Dn: 
                isnoise=1
            else:
                isnoise=-1
            near5,belong5=kdt.search(near4[i][1].data[:-1],k+1)
            neighborhood_e, dis_neighbor,dis_noise=0,0,0
            for j in range(1,k+1):
                dis_neighbor += near5[j][0]
                if list(near5[j][1].data) in Dn:
                    dis_noise +=near5[j][0]
            clean_ei= (dis_neighbor + isnoise*(dis_noise-dis_neighbor)) / (2*dis_neighbor) #纯净度
            neighborhood_e += (clean_ei*confidence_ei*different_class)/k 
        NS_e=confidence_e * neighborhood_e #噪声得分
        if NS_e > 0:
            Dn_filted.append(list(e))
           # print('Ns_e:',NS_e)  
    noise_list=noise.values.tolist()
    TP = 0
    for c in Dn_filted:
        if c in noise_list:
            TP += 1
    FP = len(Dn_filted) - TP
    FN = len(noise_list) - TP
    precision= TP / (TP + FP)
    recall= TP / (TP + FN)
    #fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
    fh= 2*precision*recall / (precision+recall)
    re = len(Dn_filted) / len(df) #移除率
    false_re= FP / len(data)#错误移除比率
    print('准确率:',precision)
    print('召回率:',recall)
    print('F0.5值:',fh)
    print('移除率:',re)
    return fh,re,false_re

In [ ]:
%%time
df=cmc_sample
data2=np.array(df.drop('label',axis=1))
dist2=[]
for m in range(0,len(data2)-1):
    for n in range(m+1,len(data2)):
        dist2.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
a = np.percentile(dist2,10) #密度阈值(%10分位数)
k=7
x1 = np.array([[1,1],[1,-1]])
x2 = np.array([1,np.floor(k / 3)/ k])
(Ps_,Pd_)=np.linalg.solve(x1,x2)
H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值

F_list,Re_list,F_list2,Re_list2,Fre_list,Fre_list2=[],[],[],[],[],[]
for rate in[0.05,0.1,0.2,0.3,0.4]:
#for rate in [0.05]:
    Fh,Re,Fh2,Re2,false_re,false_re2=[],[],[],[],[],[]
    for term in range(1): #多次试验取均值

        noise_set=pd.DataFrame(columns=df.columns)
        for r in range(0,(df.label.max()+1)):    
            noise_set=pd.concat([noise_set,df[df.label==r].sample(frac=rate, replace=False,random_state=42)]) #选取噪声比例  
        #bb=df[df.label==1].sample(frac=rate, replace=False) #选取噪声比例
        #cc=df[df.label==2].sample(frac=rate, replace=False) #选取噪声比例

        data_cut=df[~df.index.isin(noise_set.index)]
        no=[]
        for j in range(0,len(noise_set)):
            no.append(int(np.random.rand(1)[0]*(df.label.max()+1))%10)
        noise_set['label2']=no
        noise_set.reset_index(drop=True,inplace=True)
        for l in range(0,len(noise_set)):  #随机替换标签
            label_set=list(set(df.label))
            if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                label_set.remove(noise_set.loc[l,'label'])
                noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
        noise=noise_set.drop(['label'],axis=1)
        noise.rename(columns={'label2':'label'}, inplace = True)
        data= pd.concat([data_cut,noise],axis=0,ignore_index=True) #含噪声数据集
        data=shuffle(data)
        data=np.array(data)
        kdt = KdTree()
        kdt.create(data)
        
        result=run(data)
        result2=run2(data)
        Fh.append(result[0])
        Re.append(result[1])
        false_re.append(result[2])
        Fh2.append(result2[0])
        Re2.append(result2[1])
        false_re2.append(result2[2])
    F_list.append(np.mean(Fh))
    Re_list.append(np.mean(Re))
    Fre_list.append(np.mean(false_re))
    F_list2.append(np.mean(Fh2))
    Re_list2.append(np.mean(Re2))
    Fre_list2.append(np.mean(false_re2))

epoch..........: 0
高密度区噪声
[0.48484848 1.         1.         0.125      1.         0.
 0.         1.         0.         1.        ]
[0.48484848 1.         1.         0.125      1.         0.
 0.         1.         0.         1.        ] 0.0
[0.51515152 1.         1.         0.125      1.         0.
 0.         1.         0.         2.        ] 0.030303030303030276
[0.54545455 1.         1.         0.125      1.         0.
 0.         1.         0.         0.        ] 0.06060606060606055
[0.48484848 1.         1.         0.1875     1.         0.
 0.         1.         0.         2.        ] 0.0625
[0.45454545 1.         1.         0.1875     1.         0.
 0.         1.         0.         2.        ] 0.0694587909882282
[0.42424242 1.         1.         0.0625     1.         0.
 0.         1.         0.         0.        ] 0.08705943132243332
[0.57575758 1.         1.         0.125      1.         0.
 0.         1.         0.         2.        ] 0.09090909090909083
[0.57575758 1.         

高密度区噪声
[0.81818182 1.         1.         0.3125     1.         1.
 0.         1.         0.         2.        ]
[0.81818182 1.         1.         0.3125     1.         1.
 0.         1.         0.         0.        ] 0.0
[0.81818182 1.         1.         0.3125     1.         1.
 0.         1.         0.         2.        ] 0.0
[0.78787879 1.         1.         0.3125     1.         1.
 0.         1.         0.         2.        ] 0.030303030303030276
[0.78787879 1.         1.         0.3125     1.         1.
 0.         1.         0.         1.        ] 0.030303030303030276
[0.84848485 1.         1.         0.3125     1.         1.
 0.         1.         0.         1.        ] 0.030303030303030498
[0.75757576 1.         1.         0.3125     1.         1.
 0.         1.         0.         2.        ] 0.06060606060606055
[0.81818182 1.         1.         0.375      1.         1.
 0.         1.         0.         1.        ] 0.0625
[0.84848485 1.         1.         0.25       1.        

高密度区噪声
[0.48484848 1.         1.         0.125      1.         1.
 0.         1.         0.         1.        ]
[0.48484848 1.         1.         0.125      1.         1.
 0.         1.         0.         1.        ] 0.0
[0.51515152 1.         1.         0.125      1.         1.
 0.         1.         0.         0.        ] 0.030303030303030276
[0.54545455 1.         1.         0.125      1.         1.
 0.         1.         0.         2.        ] 0.06060606060606055
[0.54545455 1.         1.         0.125      1.         1.
 0.         1.         0.         1.        ] 0.06060606060606055
[0.42424242 1.         1.         0.125      1.         1.
 0.         1.         0.         2.        ] 0.06060606060606055
[0.42424242 1.         1.         0.125      1.         1.
 0.         1.         0.         2.        ] 0.06060606060606055
[0.48484848 1.         1.         0.1875     1.         1.
 0.         1.         0.         2.        ] 0.0625
[0.48484848 1.         1.         0.1875 

高密度区噪声
[0.27272727 1.         1.         0.0625     1.         1.
 0.         0.66666667 0.         0.        ]
[0.27272727 1.         1.         0.0625     1.         1.
 0.         0.66666667 0.         0.        ] 0.0
[0.3030303  1.         1.         0.0625     1.         1.
 0.         0.66666667 0.         2.        ] 0.030303030303030276
[0.24242424 1.         1.         0.0625     1.         1.
 0.         0.66666667 0.         2.        ] 0.030303030303030276
[0.27272727 1.         1.         0.125      1.         1.
 0.         0.66666667 0.         2.        ] 0.0625
[0.3030303  1.         1.         0.125      1.         1.
 0.         0.66666667 0.         0.        ] 0.0694587909882282
[0.33333333 1.         1.         0.125      1.         1.
 0.         0.66666667 0.         1.        ] 0.0870594313224334
[0.18181818 1.         1.         0.0625     1.         1.
 0.         0.66666667 0.         1.        ] 0.09090909090909083
[0.36363636 1.         1.         0.0625  

高密度区噪声
[0.75757576 1.         1.         0.25       1.         1.
 0.         1.         0.         1.        ]
[0.75757576 1.         1.         0.25       1.         1.
 0.         1.         0.         1.        ] 0.0
[0.72727273 1.         1.         0.25       1.         1.
 0.         1.         0.         2.        ] 0.030303030303030276
[0.72727273 1.         1.         0.25       1.         1.
 0.         1.         0.         1.        ] 0.030303030303030276
[0.6969697 1.        1.        0.25      1.        1.        0.
 1.        0.        1.       ] 0.06060606060606055
[0.75757576 1.         1.         0.3125     1.         1.
 0.         1.         0.         2.        ] 0.0625
[0.78787879 1.         1.         0.3125     1.         1.
 0.         1.         0.         1.        ] 0.0694587909882282
[0.78787879 1.         1.         0.3125     1.         1.
 0.         1.         0.         2.        ] 0.0694587909882282
[0.78787879 1.         1.         0.1875     1.    

...

高密度区噪声
[0.48484848 1.         1.         0.1875     1.         1.
 0.         1.         0.         0.        ]
[0.48484848 1.         1.         0.1875     1.         1.
 0.         1.         0.         0.        ] 0.0
[0.48484848 1.         1.         0.1875     1.         1.
 0.         1.         0.         0.        ] 0.0
[0.51515152 1.         1.         0.1875     1.         1.
 0.         1.         0.         1.        ] 0.030303030303030276
[0.51515152 1.         1.         0.1875     1.         1.
 0.         1.         0.         0.        ] 0.030303030303030276
[0.51515152 1.         1.         0.1875     1.         1.
 0.         1.         0.         2.        ] 0.030303030303030276
[0.45454545 1.         1.         0.1875     1.         1.
 0.         1.         0.         1.        ] 0.030303030303030276
[0.45454545 1.         1.         0.1875     1.         1.
 0.         1.         0.         2.        ] 0.030303030303030276
[0.54545455 1.         1.         0.1875

高密度区噪声
[0.60606061 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ]
[0.60606061 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ] 0.0
[0.57575758 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ] 0.030303030303030276
[0.63636364 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         0.        ] 0.030303030303030498
[0.63636364 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ] 0.030303030303030498
[0.57575758 1.         1.         0.125      0.         1.
 0.66666667 1.         0.         1.        ] 0.0694587909882282
[0.66666667 1.         1.         0.25       0.         1.
 0.66666667 1.         0.         1.        ] 0.08705943132243348
[0.6969697  1.         1.         0.125      0.         1.
 0.66666667 1.         0.         2.        ] 0.1103209536303842
[0.72727273 1.         1.   

高密度区噪声
[0.63636364 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ]
[0.63636364 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ] 0.0
[0.63636364 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         0.        ] 0.0
[0.60606061 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ] 0.030303030303030498
[0.57575758 1.         1.         0.1875     0.         1.
 0.66666667 1.         0.         2.        ] 0.060606060606060774
[0.66666667 1.         1.         0.25       0.         1.
 0.66666667 1.         0.         1.        ] 0.0694587909882282
[0.6969697  1.         1.         0.125      0.         1.
 0.66666667 1.         0.         2.        ] 0.08705943132243332
[0.57575758 1.         1.         0.125      0.         1.
 0.66666667 1.         0.         1.        ] 0.08705943132243348
[0.72727273 1.         1.         0.1875    

高密度区噪声
[0.21212121 1.         1.         0.0625     1.         1.
 0.66666667 0.66666667 0.         1.        ]
[0.21212121 1.         1.         0.0625     1.         1.
 0.66666667 0.66666667 0.         1.        ] 0.0
[0.21212121 1.         1.         0.0625     1.         1.
 0.66666667 0.66666667 0.         2.        ] 0.0
[0.24242424 1.         1.         0.0625     1.         1.
 0.66666667 0.66666667 0.         0.        ] 0.030303030303030276
[0.18181818 1.         1.         0.0625     1.         1.
 0.66666667 0.66666667 0.         2.        ] 0.030303030303030276
[0.15151515 1.         1.         0.0625     1.         1.
 0.66666667 0.66666667 0.         1.        ] 0.06060606060606066
[0.24242424 1.         1.         0.125      1.         1.
 0.66666667 0.66666667 0.         2.        ] 0.0694587909882282
[0.24242424 1.         1.         0.125      1.         1.
 0.66666667 0.66666667 0.         0.        ] 0.0694587909882282
[0.27272727 1.         1.         0.125      

高密度区噪声
[0.15151515 0.66666667 0.66666667 0.0625     1.         1.
 0.33333333 0.33333333 0.         0.        ]
[0.15151515 0.66666667 0.66666667 0.0625     1.         1.
 0.33333333 0.33333333 0.         0.        ] 0.0
[0.21212121 0.66666667 0.66666667 0.0625     1.         1.
 0.33333333 0.33333333 0.         2.        ] 0.06060606060606066
[0.15151515 0.66666667 0.66666667 0.125      1.         1.
 0.33333333 0.33333333 0.         0.        ] 0.0625
[0.15151515 0.66666667 0.66666667 0.         1.         1.
 0.33333333 0.33333333 0.         1.        ] 0.0625
[0.18181818 0.66666667 0.66666667 0.         1.         1.
 0.33333333 0.33333333 0.         1.        ] 0.06945879098822824
[0.21212121 0.66666667 0.66666667 0.125      1.         1.
 0.33333333 0.33333333 0.         0.        ] 0.0870594313224334
[0.06060606 0.66666667 0.66666667 0.0625     1.         1.
 0.33333333 0.33333333 0.         2.        ] 0.09090909090909094
[0.27272727 0.66666667 0.66666667 0.0625     1.         

In [ ]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib

In [ ]:
noise_rate=[5,10,20,30,40]
fig = plt.figure(figsize=(6,14))
ax1 = axislines.Subplot(fig, 3,1,1)
fig.add_subplot(ax1)
ax1.set_xticks(noise_rate)
ax1.axis([0,40,0,1])
plt.plot(noise_rate,F_list,'ro-',label='DDF')
plt.plot(noise_rate,F_list2,'b*-.',label='DDF-SCORE')
#plt.xlabel('噪声比例/%')
plt.ylabel('F0.5')
plt.title('不同噪声下F0.5值',fontsize=12)
plt.legend()
plt.show()
ax2 = axislines.Subplot(fig, 3,1,2)
fig.add_subplot(ax2)
ax2.set_xticks([5,10,20,30,40])
ax2.set_yticks([0,0.2,0.4,0.6,0.8])
ax2.axis([0,40,0,0.8])
plt.plot(noise_rate,Re_list,'ro-',label='DDF')
plt.plot(noise_rate,Re_list2,'b*-.',label='DDF-SCORE')
#plt.xlabel('噪声比例/%')
plt.ylabel('Re')
plt.title('不同噪声下的过滤值Re',fontsize=12)
plt.legend()

plt.show()
ax3 = axislines.Subplot(fig, 3,1,3)
fig.add_subplot(ax3)
ax3.set_xticks(noise_rate)
ax3.axis([0,40,0,0.3])
plt.plot(noise_rate,Fre_list,'ro-',label='DDF')
plt.plot(noise_rate,Fre_list2,'b*-.',label='DDF-SCORE')
plt.xlabel('噪声比例/%')
plt.ylabel('false_rate')
plt.title('不同噪声下错误移除比例',fontsize=12)
plt.legend()
plt.tight_layout(6) #设置子图间隔
plt.suptitle('(Conce)')
plt.show()